In [1]:
# 모델 2~6 으로 코드 변경시 변경해야 할 것
# PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # aram_model1.pt 여기서 1 > 2,3,4,5,6
# model1 = torch.load(PATH, map_location=device)   # mode1 > 2,3,4,5,6
# testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/Colab Notebooks/train_data/model1/test',    # 여기서 1 > 2,3,4,5,6 (폴더는 학습에서 잡혀있음)
# model1.eval()
# output = model1(data) 

# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')  
# 모델.pt파일 경로
PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # 모델1
# PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
# CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 모델1 불러오기
!pip install efficientnet_pytorch
model1 = torch.load(PATH, map_location=device)
# 모델1 파라미터 불러오기
# model1_p = torch.load(PATH2, map_location=device)
## test 이미지파일 전처리, 텐서화
import torchvision
from torchvision import transforms
# 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
transforms_test = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])
# root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/Colab Notebooks/train_data/model1/test', 
					transform = transforms_test)
## 확인
testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
# rgb three channel 이니까 3
# 높이 넓이 600 600 리사이즈
# 전처리 정상 완료 확인
# DataLoader를 통해 네트워크에 올리기
from torch.utils.data import Dataset,DataLoader 
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
# dataiter = iter(testloader)
# images = dataiter.next()
# images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
## 아웃풋, 로스, 프레딕, 아큐러시
output_list = []
model1.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
correct = 0
# 로스 연산
# import torch.nn.functional as F   # F : 테스트_로스 연산 함수
# test_loss = 0
from tqdm import tqdm # 진행률 표시를 위한

if __name__ == '__main__':
    with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
            for data, target in tqdm(testloader):                                   
                data, target  = data.to(device), target.to(device) 
                output = model1(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                output_list.append(output);
                # test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                # pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                # correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
# test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
# 아큐러시 출력 ( :.4f 소수점반올림 )
# print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
# 로스, 아큐러시 출력
# print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

# 업로드 이미지에 대한 output predict 값
# 업로드 폴더로 실행 > 아웃풋리스트에서 최근 업로드 파일을 -1인덱스로 잡은 후 아웃풋 리스트에서 -1 인덱스로 최근 업로드 파일을 잡고 argmax로 아웃풋 0 1 2 3 중 최댓값의 인덱스를 리턴

#  0:양호, 1:경증, 2:중등도, 3:중증
print(output_list[-1]) # tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0') 로 각 아웃풋 4개에 대한 수치를 알려줌
print(output_list[-1].argmax(dim=1, keepdim=True))   # 위 값 중 최댓값의 인덱스를 알려줌 > 즉 predict 값

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=5a2df34dcfa2a95f60032249a2d16ef8a226bd674eb6f7d46fb17e3c1ea19d90
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


100%|██████████| 1084/1084 [04:06<00:00,  4.40it/s]


(tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0'),
 tensor([[2]], device='cuda:0'))